# 实体对齐

In [57]:
include("../src/translatedata.jl")
include("../src/ettalign.jl")
mkpath("EA_data")

"EA_data"

In [42]:
ea_triples = [(get(dict_ILLs, sub, sub), rel, get(dict_ILLs, obj, obj)) for (sub, rel, obj) in triples]
train_ques_ner = [(que, get(dict_ILLs, ner, ner)) for (que, ner) in train_ques_rels]

14051-element Vector{Tuple{SubString{String}, AbstractString}}:
 ("what is the title leader of the bay that NER is famous for?", "Reman")
 ("what is the logo of the car that NER is related to?", "king_of_tea")
 ("which format does the cause of NER, a japanese biological warfare plan during world war ii, belong to?", "Night_Sakura")
 ("what is the parent mountain of the client of NER?", "Ball_Bearing_(advertisement)")
 ("which mountain range does the client of NER belong to?", "Cake_(advertisement)")
 ("which mountain range does the client of NER belong to?", "Changes_(advertisement)")
 ("which mountain range does the client of NER, a skyscraper in london, belong to?", "Lots_Road_South_Tower")
 ("which official language is used by the country that is affected by NER?", "1957_Mongolia_earthquake")
 ("what is the official language of the country that is affected by NER, an erdbeben?", "1957_Mongolia_earthquake")
 ("what is the leader name of the country that is affected by NER?", "262_Sou

In [58]:
dict_ILLs = ill2dict(ILLs)

Dict{String, String} with 19574 entries:
  "Naval_Moralejo"          => "Naval_Moralejo"
  "New_London_County,_Conn… => "New_London_County_(Connecticut)"
  "Priapus"                 => "Priaps"
  "Xi_County,_Henan"        => "Xixian"
  "Portwater_County_(Kansa… => "Portwater_County_(Kansas)"
  "Association_of_American… => "American_Association_of_Colleges_and_Universiti…
  "FBI_director"            => "FBI_director"
  "Tiszabercel"             => "tisauberzell"
  "Joseph_McCarthy"         => "Joseph_Redmond_McCarthy"
  "Azov_Sea"                => "Azov_Sea"
  "Chelyabinsk_Region"      => "Chelyabinsk_Region"
  "Beire_(Paredes)"         => "Berry_(Portugal)"
  "Young_Africans_SC"       => "young_african_sports_club"
  "west_jerez"              => "west_jerez"
  "Godfrey_Harold_Hardy"    => "Godfrey_Harold_Hardy"
  "2011-12_DFB_Cup"         => "2011-2012_German_FA_Cup"
  "Yves_Brechet"            => "Yves_Brecher"
  "Lightspeed_Masked_Man_2… => "Lightspeed_Masked_Man_21"
  "N-I_(rocket)

In [30]:
open("EA_data/triples.txt", "w") do io
    for (sub, rel, obj) in triples
        sub = get(dict_ILLs, sub, sub)
        obj = get(dict_ILLs, sub, sub)
        println(io, sub, '\t', rel, '\t', obj)
    end
end
open("EA_data/train_ques.txt", "w") do io
    for sol in train_ques_sols
        que, ner, rels = sol[1], sol[2], sol[3:end]
        ner = get(dict_ILLs, ner, ner)
        println(io, que, '\t', ner, '\t', join(rels, '\t'))
    end
end

open("EA_data/valid_ques.txt", "w") do io
    for (que, ner) in valid_ques_ner
        ner = get(dict_ILLs, ner, ner)
        println(io, que, '\t', ner)
    end
end

In [7]:
function sort_count(list)
    c_list = counter(list)
    sort(Dict(c_list), by=i->c_list[i], rev=true)
end

sort_count (generic function with 1 method)

In [35]:
using DataStructures
edges = DefaultDict{String, Vector{Tuple}}(Vector{Tuple})
for (sub, rel, obj) in ea_triples
    push!(edges[sub], (rel, obj))
end

In [36]:
subs = unique!(first.(ea_triples))
# 第一跳
jumps = Dict(sub => [length(edges[sub])] for sub in subs)
# 第二跳
for sub in subs
    push!(jumps[sub], sum(haskey(jumps, obj) && jumps[obj][1] for (rel, obj) in edges[sub]))
end
# 第三跳
for sub in subs
    push!(jumps[sub], sum(haskey(jumps, obj) && jumps[obj][2] for (rel, obj) in edges[sub]))
end

In [37]:
jump_three(sub) = haskey(jumps, sub) ? jumps[sub][3] : 0
jump_two(sub) = haskey(jumps, sub) ? jumps[sub][2] : 0

jump_two (generic function with 1 method)

In [43]:
fail2, fail3 = String[], String[]
for (que, ner) in train_ques_ner
    jump_two(ner) == 0 && push!(fail2, ner)
    jump_three(ner) == 0 && push!(fail3, ner)
end

In [54]:
sort(getindex.(values(jumps), 3), rev=true)

119371-element Vector{Int64}:
 4699
 3398
 2866
 2596
 2580
 2170
 2155
 2004
 1929
 1927
 1876
 1783
 1780
    ⋮
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0
    0

In [47]:
using DataStructures
function sort_count(list)
    c_list = counter(list)
    sort(Dict(c_list), by=i->c_list[i], rev=true)
end

sort_count (generic function with 1 method)